<a href="https://colab.research.google.com/github/Varfalamei/cross-domain-recsys/blob/lab3/notebooks/matching_films_and_books.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!gdown --id 1NM4uSjSg0e2LPBT_aC0v1XaRRh932jxJ
!gdown --id 19IDNSFnAh-JiVx05BSJJRmU-eTdzlG1T
!gdown --id 1iN16x8HRjjaNrqMCq-WQLZw6LQJuVdB2

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1NM4uSjSg0e2LPBT_aC0v1XaRRh932jxJ
To: /content/books.csv
100% 69.0M/69.0M [00:01<00:00, 51.5MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=19IDNSFnAh-JiVx05BSJJRmU-eTdzlG1T
To: /content/films.csv
100% 28.6M/28.6M [00:00<00:00, 183MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://

In [3]:
import pickle
import pandas as pd

In [4]:
books = pd.read_csv('books.csv')
films = pd.read_csv('films.csv')

books = books[~books.isna().any(axis=1)].reset_index(drop=True)
films = films[~films.isna().any(axis=1)].reset_index(drop=True)

In [5]:
books_to_save = books.groupby(['title']).agg({'popularity': 'max'}).reset_index()
# films_to_save = films.groupby(['title']).agg({'popularity': 'max'}).reset_index()

books = books_to_save.merge(books, on=['title', 'popularity'], how='inner')
# films = films_to_save.merge(films, on=['title', 'popularity'], how='inner')

books = books.drop_duplicates(['title', 'popularity']).reset_index(drop=True)
# films = films.drop_duplicates(['title', 'popularity']).reset_index(drop=True)

In [6]:
books_find = books[['title', 'id', 'authors']].rename(columns={"authors": "authors_book", "title": 'title_book'}).copy()
films_find = films[['title', 'id', 'year']].rename(columns={"year": "year_film", "title": 'title_film'}).copy()

books_find.title_book = books_find.title_book.str.strip().replace({"'": "", '"': "", "#": ''}, regex=True).str.lower()
films_find.title_film = films_find.title_film.str.strip().replace({"'": "", '"': "", "#": ''}, regex=True).str.lower()

matching = books_find.merge(films_find, left_on='title_book', right_on='title_film', how='inner', suffixes=('_book', '_film'))
matching

,title_book,id_book,authors_book,title_film,id_film,year_film
0,1984,7123,Джордж Оруэлл,1984,1225,1956.0
1,1984,7123,Джордж Оруэлл,1984,17791,1956.0
2,20 000 лье под водой,51824,Жюль Верн,20 000 лье под водой,4427,1954.0
3,5-я волна,1301,Рик Янси,5-я волна,15808,2016.0
4,9 жизней,33588,Олег Трегубов,9 жизней,4434,2018.0
...,...,...,...,...,...,...
2793,ярмарка тщеславия,25758,Уильям Теккерей,ярмарка тщеславия,15452,2018.0
2794,ярослав мудрый,55432,Наталья Павлищева,ярослав мудрый,22571,1981.0
2795,ярость,51412,Юрий Никитин,ярость,22825,2008.0
2796,ярость,51412,Юрий Никитин,ярость,23292,2021.0


In [7]:
with open("book_film_mapped.pkl", 'rb') as f:
  book_film_mapped = pickle.load(f)

book_film_mapped = book_film_mapped.rename(columns={'book_title': 'title_book', 'film_title':'title_film'})

book_film_mapped.title_book = book_film_mapped.title_book.str.strip().replace({"'": "", '"': "", "#": ''}, regex=True).str.lower()
book_film_mapped.title_film = book_film_mapped.title_film.str.strip().replace({"'": "", '"': "", "#": ''}, regex=True).str.lower()

book_film_mapped = book_film_mapped[book_film_mapped.title_book != book_film_mapped.title_film]

book_film_mapped = book_film_mapped[['title_book', 'book_author', 'title_film', 'film_year']].drop_duplicates()

books_for_second_check = books_find.rename(columns={'title': 'title_book', 'authors_book':'book_author', 'id': 'id_book'})
films_for_second_check = films_find.rename(columns={'title': 'title_film', 'year_film':'film_year', 'id': 'id_film'})

check = book_film_mapped.merge(books_for_second_check, on=['title_book', 'book_author'])
check = check.merge(films_for_second_check, on=['title_film', 'film_year'])
check = check[['id_book', 'id_film']]

additional_adapt = check.merge(books[['title', 'id', 'authors']], left_on='id_book', right_on='id')
additional_adapt = additional_adapt.drop(['id'], axis=1)

additional_adapt = additional_adapt.merge(films[['title', 'id', 'year']], left_on='id_film', right_on='id')
additional_adapt = additional_adapt.drop(['id'], axis=1)

additional_adapt = additional_adapt.rename({'title_x': 'title_book', 'authors': 'authors_book', 'title_y': 'title_film', 'year': 'year_film'}, axis=1)


In [8]:
matching = pd.concat([matching, additional_adapt], axis=0)
matching.shape

(3505, 6)

In [9]:
matching.drop_duplicates()

,title_book,id_book,authors_book,title_film,id_film,year_film
0,1984,7123,Джордж Оруэлл,1984,1225,1956.0
1,1984,7123,Джордж Оруэлл,1984,17791,1956.0
2,20 000 лье под водой,51824,Жюль Верн,20 000 лье под водой,4427,1954.0
3,5-я волна,1301,Рик Янси,5-я волна,15808,2016.0
4,9 жизней,33588,Олег Трегубов,9 жизней,4434,2018.0
...,...,...,...,...,...,...
702,Мэри Поппинс,34878,Памела Трэверс,"Мэри Поппинс, до свидания",15212,1983.0
703,Мэри Поппинс и дом по соседству,7931,Памела Трэверс,"Мэри Поппинс, до свидания",15212,1983.0
704,Отцы и дети. Ася. Повести. Рассказы,53434,Иван Тургенев,Отцы и дети,20356,1959.0
705,Питер Пэн,4059,Джеймс Барри,Питер Пэн: Возвращение в Нетландию,17647,2002.0


In [ ]:
# нужно приджойнить к книгам из файла Эмиля айдиншники второго файла эмиля

In [210]:
matching.to_csv('film_adaptations.csv', index=False)

In [13]:
matching = matching.rename({'id_book': 'book_id', 'id_film': 'film_id'}, axis=1)

In [21]:
from typing import Dict, List

def metric_film_adaptations(
    recommendations: pd.DataFrame,
    matching: pd.DataFrame
) -> float:
    """
    This metric is calculated based on movie recommendations based on books.
    We want to know if we have a movie adaptation of the book in our recommendation.

    How to calculate:
      If there is a film adaptation of the book, we put a match, otherwise there 
      is no match. If there aren't any film adaptations we add 0.5 to result.
      Next, we average the result

    Explanation of the metric:
      This metric like a roc_auc. If metric > 0.5, it is good, otherwise it is bad XD

    Input:
      recommendations: Dataframe with book id as a key and movie ids as value
      matching: Dataframe that contains a book id and a movie id.
                Has to have 'id_book' and 'id_film' columns

    Return:
      metric value
    """

    assert 'book_id' in set(recommendations.columns) and \
      'film_id' in set(recommendations.columns) and \
      'book_id' in set(matching.columns) and \
      'film_id' in set(matching.columns), \
      "matching doesn't contain right columns, please rename them to id_film and id_book"
    book_ids = list(recommendations.book_id.unique())
    result = 0

    for book_id in book_ids:
        film_recs = recommendations[recommendations.book_id == book_id].film_id.tolist()
        film_adaptations = matching[matching.book_id == book_id].film_id.tolist()
        if film_adaptations != []:
            if set(film_adaptations).intersection(set(film_recs)) != set():
                result += 1
            else:
                result += 0
        else:
            result += 0.5

    return round(result / len(book_ids), 4)



In [22]:
metric_film_adaptations(matching, matching)

1.0

In [16]:
matching.book_id.unique()

array([ 7123, 51824,  1301, ...,  7931, 53434, 52852])

[1225, 17791]